<a href="https://colab.research.google.com/github/OjasKhandelwal/LearnML/blob/main/ANN_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.optim as optim

In [2]:
#set random seeds for reproductibilty
torch.manual_seed(42)

In [4]:
df = pd.read_csv('/content/fashion-mnist_train.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#train test split

x = df.iloc[: , 1:].values
y = df.iloc[: , 0].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#scaling the features
x_train =  x_train/255.0
x_test =  x_test/255.0

In [23]:
#create CustomDataset class

class CustomDataset(Dataset):
  def __init__(self , features ,labels):
    self.features  = torch.tensor(features , dtype=torch.float32)
    self.labels = torch.tensor(labels , dtype=torch.long)

  def __len__(self):
    return len(self.features)


  def __getitem__(self , index):
    return self.features[index] , self.labels[index]

In [27]:
#create train_dataset object
train_dataset = CustomDataset(x_train , y_train)

#test_dataset object
test_dataset = CustomDataset(x_test , y_test)

#create train & test loader
train_loader = DataLoader(train_dataset , batch_size=32 , shuffle=True)
test_loader = DataLoader(test_dataset , batch_size=32 , shuffle=False)

In [40]:
#create nn class

class MyNN(nn.Module):
  def __init__(self , num_features):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features , 128),
        nn.ReLU(),
        nn.Linear(128 , 64),
        nn.ReLU(),
        nn.Linear(64 , 10)

    )



  def forward(self , x):
    return self.model(x)


In [41]:
#set learning rate & epochs
epochs = 100
learning_rate = 0.1

#instantiate the model
model = MyNN(x_train.shape[1])

#loss function
loss_fn = nn.CrossEntropyLoss()

#optimizer
optimizer = optim.SGD(model.parameters() , lr=learning_rate)



In [42]:
#training loop

for epoch in range(epochs):
  for batch_features , batch_labels in train_loader:

    #forward pass
    outputs = model(batch_features)

    #calculate loss
    loss = loss_fn(outputs , batch_labels)

    #backward pass
    optimizer.zero_grad()
    loss.backward()

    #update weights
    optimizer.step()


In [43]:
model.eval( )

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [46]:
#evaluation code

total = 0
correct = 0

with torch.no_grad():
  for batch_features , batch_labels in test_loader:
    outputs = model(batch_features)
    _, predicted = torch.max(outputs , 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()


print(correct/total)

0.8853333333333333
